In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

## Code for extracting "impression" section using regular expression. 

In [365]:
df_pn = pd.read_csv("data/Potential_negative.csv")
df_pp = pd.read_csv("data/Potential_positive.csv")

In [398]:

impressions = []
bad_list = ['Findings were discussed','Findings discussed','Findings reported','Findings communicated',
            'findings were communicated','The above findings','Please contact the interpreting',
            'findings of the study','via telephone','recommendations were discussed','information was discussed',
            'discussed with Dr.','communicated to Dr.','reviewed with Dr.','called with these results',
            'was notified','Findings were communicated','findings were conveyed to',
            'Up-to-date CT equipment','If this radiology report', 'DLP:','CTDIvol:','have reviewed the examination',
            'Red Document','LS1R','WY2R','Fleischner','Fleischer','Resident/Fellow',
            'PowerConnect', 'PowerScribe','Workstation ID','1H',
            'Note: These guidelines do not apply','For comments and reference','For information about',
            'See below recommendations','100','80','UN1','(Incidental alert)','Findings were conveyed','Message ID',
            '____','US2','UN2','Receipt of this communication by','findings were discussed','discussed with',
            'call ED radiology', 'reported to Dr.','Orange Alert', 'Orange alert','called with these findings',
            'ED message','Please see below','Please see separate report','----','guidelines for follow-up'
            ]


def extract(df):
    impressions = []
    for index, row in df.iterrows():
        an = row['Accession Number']
        sex = row['Patient Sex']
        age = row['Patient Age']
        date = row['Ordered Date']
        report = row['Report Text']
        start = '(IMPRESSION:|Impression:)' #HOW THE IMPRESSION SECTION STARTS

        main = '((.|\n|\r)+?)' #ALL THE CONTENT INSIDE
        end = '(\n[A-Z][A-Z]+:|$|\n(?<![A-Z\d])(?!1\d{3}\b)[A-Z\d]{4,}(?![A-Z\d])|A: Solid Nodules|B: Subsolid Nodules|<6 mm|Reference:)' #HOW IMPRESSION SECTION ENDS
        m = re.search(start + main+end, report)
        if m:
            found = m.group(2)
            print("@@@@@@@@", index, "@@@@@@@")
#             print(found)
            print(" ")

            #get rid of extra whitespaces, extra newlines, tabs, etc. 

            found = found.strip() #gets rid of newlines on its own.
            found = re.sub(' +', ' ', found) 
            found = re.sub(r'(\n\s\r*)+\n', '\n', found)
            found = re.sub(r'\n \n','\n',found)
            found = re.sub(r'\n+','\n',found)
            found = re.sub(r'\t',' ',found)

            #MORE DATA CLEANING
    #         found = re.sub(r'\n_+\n','\n',found)
    #         found = re.sub(r'\n_+$','',found)

#             found = re.sub(r'(?<![!.,])\n', '.\n', found)
            found = found.strip()

            found = nltk.sent_tokenize(found)

            filtered_list = []

            for sent in found:
                if not any(x in sent for x in bad_list) and not sent == '.' and not sent == 'Single.' and not sent == 'Multiple.' and not sent == 'Multiple' and not sent == 'Single':
                    filtered_list.append(sent)
            
            print(filtered_list[-1])
            
            found = "\n".join(filtered_list)
#             print(found)
            impressions.append([an,found, sex, age, date])
        else:
            print("ERROR")
    return impressions

In [2]:
# pn_impressions = extract(df_pn)
# pp_impressions = extract(df_pp)

In [1]:
# pn_impressions

In [354]:
df_pn_impressions = pd.DataFrame(pn_impressions, columns=['Accession Number', 'Impression', 'Parient Sex', 'Patient Age', 'Ordered Date'])
df_pn_impressions = df_pn_impressions.sample(frac=1)
df_pp_impressions = pd.DataFrame(pp_impressions, columns=['Accession Number', 'Impression', 'Parient Sex', 'Patient Age', 'Ordered Date'])
df_pp_impressions = df_pp_impressions.sample(frac=1)

In [355]:
df_pn_impressions.to_csv("data/potential_negative_impressions.csv", index=False)
df_pp_impressions.to_csv("data/potential_positive_impressions.csv", index=False)

## Creating txt files

In [130]:
for index, row in df_pn_impressions.iterrows():
    an = row['Accession Number']
    f = open('pn_impression_txt_files/'+str(an)+'_imp.txt', 'w')
    f.write(row['Report Text'])
    f.close()